In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python data audit example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [182]:
df = spark.read.format('com.databricks.spark.csv') \
            .options(header='true', inferschema='true') \
            .load("../data/bank.csv",header=True);
df.drop('day','month','poutcome').show(5)

+---+-----------+-------+---------+-------+-------+-------+----+--------+--------+--------+-----+--------+---+
|age|        job|marital|education|default|balance|housing|loan| contact|duration|campaign|pdays|previous|  y|
+---+-----------+-------+---------+-------+-------+-------+----+--------+--------+--------+-----+--------+---+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular|      79|       1|   -1|       0| no|
| 33|   services|married|secondary|     no|   4789|    yes| yes|cellular|     220|       1|  339|       4| no|
| 35| management| single| tertiary|     no|   1350|    yes|  no|cellular|     185|       1|  330|       1| no|
| 30| management|married| tertiary|     no|   1476|    yes| yes| unknown|     199|       4|   -1|       0| no|
| 59|blue-collar|married|secondary|     no|      0|    yes|  no| unknown|     226|       1|   -1|       0| no|
+---+-----------+-------+---------+-------+-------+-------+----+--------+--------+--------+-----+--------+---+
o

AttributeError: 'DataFrame' object has no attribute 'transpose'

In [125]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [126]:
types = [f.dataType for f in df.schema.fields]

In [127]:
df.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string')]

In [128]:
out = spark.createDataFrame(df.dtypes).toDF('Names','Types')

In [129]:
out.show()

+---------+------+
|    Names| Types|
+---------+------+
|      age|   int|
|      job|string|
|  marital|string|
|education|string|
|  default|string|
|  balance|   int|
|  housing|string|
|     loan|string|
|  contact|string|
|      day|   int|
|    month|string|
| duration|   int|
| campaign|   int|
|    pdays|   int|
| previous|   int|
| poutcome|string|
|        y|string|
+---------+------+



In [130]:
df.select('job').describe().show()

+-------+-------+
|summary|    job|
+-------+-------+
|  count|   4521|
|   mean|   null|
| stddev|   null|
|    min| admin.|
|    max|unknown|
+-------+-------+



In [171]:
Quantiles = df.stat.approxQuantile('age',(0.25,0.5,0.75),0.0)
Quantiles

[33.0, 39.0, 49.0]

In [175]:
from pyspark.sql.functions import mean, min, max, col
from pyspark.sql.types import *


# get variable name and types
out = spark.createDataFrame(df.dtypes).toDF('Names','Types')

schema = StructType([
    StructField("mean", LongType(), True), 
    StructField("min", LongType(), False), 
    StructField("max", LongType(), False)])

schema_freq = StructType([
    StructField("freqItems", LongType(), True)])

df_stats = spark.createDataFrame([],schema)
df_freq = spark.createDataFrame([],schema_freq)


for i in df.columns:

    des_d = df.select([mean(i).alias('mean'), min(i).alias('min'), max(i).alias('max')])
    #des_d.printSchema()
    freq = df.stat.freqItems([i], 0.3)
#     Quantiles = df.stat.approxQuantile(i,(0.25,0.5,0.75),0.0)
#     print(Quantiles)
    freq = freq.withColumn('freqItems',col(i+'_freqItems')).select('freqItems')
    freq.show(1,False)
    #freq.printSchema()
    
    df_stats = df_stats.union(des_d)
    #df_freq = df_freq.union(freq)
 

+---------+
|freqItems|
+---------+
|[57]     |
+---------+

+---------------------------------------+
|freqItems                              |
+---------------------------------------+
|[management, entrepreneur, blue-collar]|
+---------------------------------------+

+---------------------------+
|freqItems                  |
+---------------------------+
|[married, divorced, single]|
+---------------------------+

+------------------------------+
|freqItems                     |
+------------------------------+
|[tertiary, secondary, primary]|
+------------------------------+

+---------+
|freqItems|
+---------+
|[no, yes]|
+---------+

+---------+
|freqItems|
+---------+
|[1136]   |
+---------+

+---------+
|freqItems|
+---------+
|[no, yes]|
+---------+

+---------+
|freqItems|
+---------+
|[no, yes]|
+---------+

+------------------------------+
|freqItems                     |
+------------------------------+
|[cellular, telephone, unknown]|
+------------------------------+

+

In [140]:
df_stats.show()

+------------------+--------+-------+
|              mean|     min|    max|
+------------------+--------+-------+
| 41.17009511170095|      19|     87|
|              null|  admin.|unknown|
|              null|divorced| single|
|              null| primary|unknown|
|              null|      no|    yes|
|1422.6578190665782|   -3313|  71188|
|              null|      no|    yes|
|              null|      no|    yes|
|              null|cellular|unknown|
|15.915284229152842|       1|     31|
|              null|     apr|    sep|
|263.96129174961294|       4|   3025|
| 2.793629727936297|       1|     50|
|39.766644547666445|      -1|    871|
|0.5425790754257908|       0|     25|
|              null| failure|unknown|
|              null|      no|    yes|
+------------------+--------+-------+



In [158]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

def unequal_union_sdf(sdf1, sdf2):
    s_df1_schema = set((x.name, x.dataType) for x in sdf1.schema)
    s_df2_schema = set((x.name, x.dataType) for x in sdf2.schema)

    for i,j in s_df2_schema.difference(s_df1_schema):
        sdf1 = sdf1.withColumn(i,F.lit(None).cast(j))

    for i,j in s_df1_schema.difference(s_df2_schema):
        sdf2 = sdf2.withColumn(i,F.lit(None).cast(j))

    common_schema_colnames = sdf1.columns
    sdk = \
        sdf1.select(common_schema_colnames).union(sdf2.select(common_schema_colnames))
    return sdk 

In [159]:
sdf_concat = unequal_union_sdf(out,df_stats)

In [160]:
sdf_concat.show()

+---------+------+-----------------+-------+--------+
|    Names| Types|             mean|    max|     min|
+---------+------+-----------------+-------+--------+
|      age|   int|             null|   null|    null|
|      job|string|             null|   null|    null|
|  marital|string|             null|   null|    null|
|education|string|             null|   null|    null|
|  default|string|             null|   null|    null|
|  balance|   int|             null|   null|    null|
|  housing|string|             null|   null|    null|
|     loan|string|             null|   null|    null|
|  contact|string|             null|   null|    null|
|      day|   int|             null|   null|    null|
|    month|string|             null|   null|    null|
| duration|   int|             null|   null|    null|
| campaign|   int|             null|   null|    null|
|    pdays|   int|             null|   null|    null|
| previous|   int|             null|   null|    null|
| poutcome|string|          

In [38]:
df.crosstab('age','default').show()

+-----------+---+---+
|age_default| no|yes|
+-----------+---+---+
|         69|  6|  0|
|         56| 72|  2|
|         42|138|  3|
|         24| 23|  1|
|         37|158|  3|
|         25| 43|  1|
|         52| 86|  0|
|         20|  3|  0|
|         46|119|  0|
|         57| 87|  4|
|         78|  3|  0|
|         29| 97|  0|
|         84|  1|  0|
|         61| 16|  0|
|         74|  3|  0|
|         60| 47|  0|
|         28|102|  1|
|         38|158|  1|
|         70|  7|  0|
|         21|  7|  0|
+-----------+---+---+
only showing top 20 rows



In [177]:
df = spark.createDataFrame([(1, 2, 3) if i % 2 == 0 else (i, 2 * i, i % 4) for i in range(100)], ["a", "b", "c"])
df.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
|  1|  2|  3|
| 11| 22|  3|
|  1|  2|  3|
| 13| 26|  1|
|  1|  2|  3|
| 15| 30|  3|
|  1|  2|  3|
| 17| 34|  1|
|  1|  2|  3|
| 19| 38|  3|
+---+---+---+
only showing top 20 rows



In [180]:
# Given the above DataFrame, the following code finds the
# frequent items that show up 40% of the time for each column:
freq = df.stat.freqItems(["a", "b", "c"], 0.2)
freq.show()

+-------------------+--------------------+-----------+
|        a_freqItems|         b_freqItems|c_freqItems|
+-------------------+--------------------+-----------+
|[23, 59, 47, 71, 1]|[2, 142, 94, 46, ...|     [1, 3]|
+-------------------+--------------------+-----------+



In [181]:
df.groupBy('a').count().sort('count', ascending=False).show()

+---+-----+
|  a|count|
+---+-----+
|  1|   51|
|  5|    1|
| 29|    1|
| 19|    1|
| 57|    1|
| 43|    1|
| 31|    1|
|  7|    1|
| 77|    1|
| 25|    1|
| 39|    1|
| 95|    1|
| 71|    1|
|  9|    1|
| 27|    1|
| 63|    1|
| 51|    1|
| 17|    1|
| 79|    1|
| 41|    1|
+---+-----+
only showing top 20 rows



In [110]:
firstDF = spark.range(3).toDF("myCol")
firstDF.show()
newRow = spark.createDataFrame([[20]])
newRow.show()
appended = firstDF.union(newRow)
appended.show()

+-----+
|myCol|
+-----+
|    0|
|    1|
|    2|
+-----+

+---+
| _1|
+---+
| 20|
+---+

+-----+
|myCol|
+-----+
|    0|
|    1|
|    2|
|   20|
+-----+



In [164]:
from numpy import floor
import time

def quantile(rdd, p, sample=None, seed=None):
    """Compute a quantile of order p ∈ [0, 1]
    :rdd a numeric rdd
    :p quantile(between 0 and 1)
    :sample fraction of and rdd to use. If not provided we use a whole dataset
    :seed random number generator seed to be used with sample
    """
    assert 0 <= p <= 1
    assert sample is None or 0 < sample <= 1

    seed = seed if seed is not None else time.time()
    rdd = rdd if sample is None else rdd.sample(False, sample, seed)

    rddSortedWithIndex = (rdd.
        sortBy(lambda x: x).
        zipWithIndex().
        map(lambda (x, i): (i, x)).
        cache())

    n = rddSortedWithIndex.count()
    h = (n - 1) * p

    rddX, rddXPlusOne = (
        rddSortedWithIndex.lookup(x)[0]
        for x in int(floor(h)) + np.array([0L, 1L]))

    return rddX + (h - floor(h)) * (rddXPlusOne - rddX)

In [ ]:
df.approxQuantile()